# A first example of a numerical simulation

In this exercise, you will conduct a first instance of numerical simulation of the time evolution of a system. We will solve an advection equation of the form

$$\frac{\partial u}{\partial t} + a \frac{\partial u}{\partial x} = 0  \tag{1}$$

with $a$ being a __constant__. In subsequent exercises, more general cases will be considered, with $a$ no longer constant.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nm_lib import nm_lib as nm, utils as utils
from importlib import reload
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
    
# Define some simple global matplotlib settings
plt.rcParams.update(
    {"font.size" : "15",
     "font.family"          : "STIXGeneral",
     "mathtext.fontset"     : "stix",
     "figure.autolayout"    : "True",
     "figure.figsize"       : (7,7),
     "lines.markersize"     : 10,
     }
)

def get_uu_t0(xx: np.ndarray) -> np.ndarray:
    nom = np.cos(6*np.pi*xx/5)**2
    den = np.cosh(5*xx**2)
    return nom/den

if __name__ == "__main__":
    _x0 = -2.6
    _xf = 2.6
    _nint = int(64)
    _xx, _dx = utils.get_xx(_nint, _x0, _xf)
    _uu_t0 = get_uu_t0(_xx)
    fig, ax = plt.subplots()
    ax.plot(_xx, _uu_t0)
    ax.set_ylabel("$u(x,t=t_0)$")
    ax.set_xlabel("$x$");

## 1 – The $a = const$ case: the mathematical problem

We would like to solve equation (1) numerically for $x  [x_0, x_f]$ with $x_0 = −2.6$, $x_f = 2.6$, periodic boundary conditions and with the initial condition:

$$u(x,t=t_0) = \cos^2 \left(\frac{6 \pi x}{5} \right) / \cosh(5x^2) \tag{2}$$

We start by representing that function in the given interval.

## 2 – Spatial derivative.

We discretize the initial condition by subdividing the spatial domain into $nint=64$ equal intervals. The function will therefore be sampled with $nump=65$ points, namely $(u_0, u_1, u_2, ... , u_{nump−1})$ at equidistant values of the abscissa, $(x_0, x_1, x_2, ..., x_{nump−1})$. Let us calculate the spatial derivative of the function through non-centered finite differences of the form:

$$\left(\frac{\partial u}{\partial x}\right)_{x=x_i} \rightarrow \frac{u_{i+1}-u_{i}}{\Delta x}  \tag{3}$$

On the basis of the experience gained with the previous batch of exercises (ex. 1), which order of approximation can we expect now for the calculation of the derivative since we are using non-centered finite differences?

### Solution
The finite difference on this form is the downwind differencing, which we now know is of 2nd order _if evaluated on half integer grid points_. For non-centered downwind the approximation is of first order. I can test this using `order_conv` from `nm_lib`, which results in a mean order of convergence $\sim1$ (again, I haven't figured out why the result of `order_conv` is negative, but I interpret the result as positiv.). For completeness I test with the 2nd and 4th order _central_ differencing, which also gives the expected result of $\sim 2$ and $\sim 4$.

<span style="color:green">JMS</span>.

<span style="color:blue">GREAT JOB! I put in exercise ex_1a_ext the following: "If I'm not mistaken the -2 is because it comes from $1/dx^2$. Does it make sense to you?" </span>.

> JB: Of course! Thank you, I didn't catch the comment on ex_1a_ext at first. But yeah, this make sense, thank you!

In [ ]:
def get_order_spatial(min_nint: int, deriv_type: str = "upw", u_of_x: callable = get_uu_t0, x0=-2.6, xf=2.6, plot: bool = True) -> np.ndarray:
    if deriv_type == "upw":
        ddx = lambda xx, uu: nm.deriv_upw(xx, uu, method="roll")
    elif deriv_type == "dnw":
        ddx = lambda xx, uu: nm.deriv_dnw(xx, uu, method="roll")
    elif deriv_type == "cent":
        ddx = lambda xx, uu: nm.deriv_cent(xx, uu, method="roll")
    elif deriv_type == "4tho":
        ddx = lambda xx, uu: nm.deriv_4tho(xx, uu, method="roll")
    else:
        print("Found no fitting deriv type")
        return None
    nints = [min_nint, 2*min_nint, 4*min_nint]
    dudx_list = []
    fig, ax = plt.subplots(nrows=2, sharex=True) if plot else (None, None)
    for nint in nints:
        xx, _ = utils.get_xx(nint, x0, xf)
        uu = u_of_x(xx)
        dudx_list.append(ddx(xx, uu))
        if plot:
            ax[0].set_title(deriv_type)
            ax[0].plot(xx, dudx_list[-1],label=f"{nint:d}")
    m = utils.order_conv(*dudx_list)
    mmean = np.nanmean(m)
    if plot:
        xx, _ = utils.get_xx(min_nint, x0, xf)
        ax[0].legend()
        ax[0].set_ylabel("$u'(x)$")
        ax[1].plot(xx, m, label="m")
        ax[1].plot(xx, mmean*np.ones_like(xx), label=f"mean = {mmean:.3g}")
        ax[1].legend()
        ax[1].set_xlabel("$x$")
        ax[1].set_ylabel("order of convergence")
    return m

if __name__ == "__main__":
    get_order_spatial(_nint,"dnw")
    get_order_spatial(_nint,"cent")
    get_order_spatial(_nint,"4tho")


## 3 – Time advance.

We want to calculate an approximation to the value of the function at times later than $t = t_0$, for instance, at time $t_0 + \Delta t$. To that end, we carry out a discretization of the time axis, calculating approximate values for the time derivative as follows:

$$\left(\frac{\partial u}{\partial t}\right)_{\underset{t=t_0}{x=x_i}} \rightarrow \frac{u_i(t_0 + \Delta t)-u_i(t_0)}{\Delta t}\tag{4}$$

Using now the differential equation (1) and expression (3), we can calculate an approximate value for the function $u$ at $x_i$ and time $t_0 + \Delta t$ as follows:

$$u_i(t_0+\Delta t) = u_i(t_0) - a \frac{u_{i+1}(t_0) - u_i(t_0)}{\Delta x}\Delta t  \tag{5}$$

Using (5), calculate $u_i$ at time $t_0 + \Delta t$ at all points of the sample excluding the rightmost point (i.e., excluding $x_{nump−1}$). Use  $\Delta t = 0.98 \Delta x/|a|$ and a = −1. Fill in `nm_lib` the functions `step_adv_burgers` and `cfl_adv_burger`. 

In [ ]:
if __name__ == "__main__":
    # This code segment calculates u(t0 + dt):
    a = -1
    xx, _ = utils.get_xx(_nint, _x0, _xf)
    uu_t0_i = get_uu_t0(xx)
    dt, uu_step = nm.step_adv_burgers(xx, uu_t0_i, a)
    uu_t0_1 = uu_t0_i + uu_step * dt

    # This loop calculates the u(t0 + i*dt) for i =(1,30) just to see what happens when
    # the solution reaches the border without periodic boundaries.
    uu_t0 = uu_t0_i
    nt = 30
    for i in range(1, nt+1):
        dt, uu_step = nm.step_adv_burgers(xx, uu_t0, a)
        uu_t1 = uu_t0 + uu_step * dt
        uu_t0 = uu_t1

    fig, ax = plt.subplots()
    ax.plot(xx, uu_t0_i, label="$u_i(t_0)$")
    ax.plot(xx, uu_t0_1, label="$u_i(t_0 + \Delta t)$")
    ax.plot(xx, uu_t1, label=f"$u_i(t_0 + {i:d}\Delta t)$")
    ax.legend()
    ax.set_ylabel("$u(x,t)$")
    ax.set_xlabel("$x$")

### Comments
Here I use the `step_adv_burgers` method to find the step in time to approximate the solution $u(x_i,t+dt)$ using the `deriv_dnw` method without periodic boundaries. The orange curve displays this solution, and we see it shifted along the x-axis to the left as expected with `a=-1`. Here I also continued the evolution of 30 more time steps, to have the solution travel beyond the end of the interval to see what happens. Of course we loose sight of the "wave" as it travels beyond `x_0`, but we can also see some residual effect on the right of the interval which I expect is due to the last ill calculated grid point that comes from using the downwind method.

## 4 – The boundaries.

To calculate the function at the rightmost point of the interval, we use a periodicity condition on $u$: $u_{nump−1}(t_0 + \Delta t) = u_0(t_0 + \Delta t)$. Consider cutting the ill-calculated (or missing) grid points and using `NumPy.pad` to add different boundary conditions. Use `wrap`

In [ ]:
if __name__ == "__main__":
    a = -1
    xx, _ = utils.get_xx(_nint, _x0, _xf)
    uu_t0_i = get_uu_t0(xx)
    uu_t0 = uu_t0_i
    nt = 30
    for i in range(nt):
        # Passing bnd_limits to step_adv_burgers:
        dt, uu_step = nm.step_adv_burgers(xx, uu_t0, a, bnd_limits=[0,1])
        uu_t1 = uu_t0 + uu_step * dt
        uu_t0 = uu_t1

    fig, ax = plt.subplots()
    ax.plot(xx, uu_t0_i, label="$u_i(t_0)$")
    ax.plot(xx, uu_t1, label=f"$u_i(t_0 + {nt:d}\Delta t)$")
    ax.legend()
    ax.set_ylabel("$u(x,t)$")
    ax.set_xlabel("$x$")

### Comments
Here I've repeated the same calculation as in the previous task, but included periodic boundaries. I've done this in the `step_adv_burgers` method, by passing in the optional keyword `bnd_limits` which replaces the appropriately ill calculated grid point in the derivative with what it should have been using periodic boundaries using `np.pad` and the `wrap` method.

## 5 – Subsequent steps in time.

Having calculated the values $u_i$, $i = 0, 1, 2, ...,nump−1,$ at time $t_0+\Delta t$, we can carry out another step in time of size $\Delta t$, following exactly the method just explained. In general, if we have calculated the value of the $u$ function at $x_i$ and time $n\Delta t$, which we will call $u^n_i$, we can carry out the next step in time, of size $\Delta t$, through the expression:

$$u_i^{n+1} = u_i^n - a \frac{u_{i+1}^n - u_i^n}{\Delta x}\Delta t  \tag{6}$$

Periodic boundary conditions must be applied, as explained in 4 above, to finish the calculation at each timestep. Fill in nm_lib the function `evolv_adv_burgers`. 

Carry out many steps in time so you can clearly understand the mathematical nature of the solution of the equation. For example, in Python, you can _matplotlib.animation_ to see the evolution. 

In [ ]:
if __name__ == "__main__":
    nt = 200
    xx, _ = utils.get_xx(64, -2.6, 2.6)
    u0 = get_uu_t0(xx)
    t, ut = nm.evolv_adv_burgers(xx, u0, nt, a=-1,
                                   ddx = lambda x,y: nm.deriv_dnw(x,y, method="roll"),
                                   bnd_limits=[0,1])
    ut = ut.T
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
    def init(): 
        axes.plot(xx,ut[:,0])

    def animate(i):
        axes.clear()
        axes.plot(xx,ut[:,i])
        axes.set_title('t=%.2f'%t[i])
        axes.set_ylim(-0.01,1.01)
        
    anim = FuncAnimation(fig, animate, interval=50, frames=nt, init_func=init)
    html = HTML(anim.to_jshtml())
    display(html)
    plt.close()

### Comments
Here I've developed the `evolve_adv_burgers` method in `nm_lib`, evolving the burgers equation `nt=200` steps forward in time. The evolve method takes in the same type of parameter `bnd_limits` and `bnd_type` which is passed through to the `step_adv_burgers` method. As far as I've understood (and tested) the periodicity of the problem is dealt with solely in the part of the evolution which is calculating the time step (the output of `step_adv_burgers`), which translates the periodic boundaries into the solution at the next time step, so with this approach I don't have to pad the actual solution `u_j^n+1`. I hope this is still the case for more complex problems, e.i. for non-uniform advection speeds. I guess I'll find out in the next exercises, but also appreciate any pointers/feedback on this way of dealing with the boundaries.

As we can see the solution evolves relatively nicely in time, but with some smearing of the local maxima and minima. Over time, the solution looses some amplitude, and the "small bumbs" on each side is more and more smeared out. This is more or less expected, as with this low resolution in space we have some very pointy peaks and troughs which will be smeared out by the neighboring points due to round off (and truncation?) errors.

<span style="color:green">JMS</span>.

<span style="color:blue">GREAT JOB!</span>.

<span style="color:blue">But the "smearing" do not come from round off error or truncation in the operations. This come from the fact that we are discretizacing the domain. Which means that we are solving the derivative using a Talyor series and as a result the "left" over from that discretization behaives as a diffusion. </span>.

> JB: Right! Thank you for clearing that up for me.

## 6 – Comparison with the exact solution.

Through one of the theory sessions, we know the exact solution of eq. (1) for the initial condition (2). Draw the exact solution on top of the numerical one using a dashed line. Then, explain the mathematical behavior of the solution. __Important note__: consider that the initial condition is eq (2) in $[x_0, x_f]$, _with periodic conditions at the boundaries_. In other words, the initial condition consists of an infinite repetition of the function (2) you represented between $[x_0, x_f]$ next to each other. Use this fact when comparing your numerical solution with the analytical one. __Hint__: if you consider points starting within the interval $(x_0, x_f)$ and moving with speed $a$, they will go outside the domain after some time. Use mod operator (\% in Python) to bring them back into the domain respecting the periodicity of the problem or _numpy.pad_ to pad ghost points, i.e., points out of the numerical domain, which will allow defining the boundaries, at both ends of the numerical domain. NumPy.pad allows various types of boundaries.

In [ ]:
def get_u_exact(xx: np.ndarray,t: float,a: float) -> np.ndarray:
    """
    Calculates the exact solution of the advective burgers equation with constant
    advection speed at time t, with periodic boundaries in x.

    Arguments:
        xx {np.ndarray} -- discretized interval of x-values
        t {float} -- time coordinate to evaluate the solution for
        a {float} -- advection speed (could also be an array)

    Returns:
        np.ndarray -- the exact solution at time t 
    """
    xmin = xx[0]    # pick out min x value, could be positiv or negative. Used to shift the x-array to start at x_0 = 0
    xmax = xx[-1]   # pick out max x value to calculate the length of the interval
    x_start_0 = xx - xmin   # new x-array with the interval shifted to so the first element is zero
    # y = x - at, but here I use the new shifted x-array so I can use the modulo operator for periodic boundaries
    y = (x_start_0-a*t)%(xmax-xmin) + xmin  # after calculating, shift the y-array back to match with the original interval
    return get_uu_t0(y)

if __name__ == "__main__":
    nint = 64
    nt = 60
    a = -1
    x0 = -2.6
    xf = 2.6
    xx, _ = utils.get_xx(nint, x0, xf)
    uu_t0 = get_uu_t0(xx)
    tt, uunt = nm.evolv_adv_burgers(xx, uu_t0, nt, a, ddx = lambda x,u:nm.deriv_dnw(x,u,method="roll"), bnd_limits=[0,1])

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

    def init(): 
        ax.plot(xx,uunt.T[:,0])

    def animate(i):
        ax.clear()
        ax.plot(xx,uunt.T[:,i], label="uunt")
        ax.plot(xx, get_u_exact(xx, tt[i],a),ls="--", label="exact")
        ax.set_title('t=%.2f'%tt[i])
        ax.legend(loc=1)
            
    anim = FuncAnimation(fig, animate, interval=100, frames=nt, init_func=init)
    html = HTML(anim.to_jshtml())
    display(html)
    plt.close()



### Comments
See docstring and comments on the periodic boundaries for the analytical solution. Here it is very easy to see the smearing effect I discussed in the previous task when we compare the solution to the exact one. Here I've actually widened the upper boundary for the x-interval to demonstrate the solution on an antisymmetric grid, which means I've even poorer resolution in space as we can see from the discretization of the curves.

I've actually discussed how the exact solution evolves in time in `ex_2a_analytical`, as I mistakenly did the first task. I refer to that notebook for a slightly more in depth analysis, but the main take away is that the shape of the solution stays constant in time, but moving in the direction of the sign of `a` with a constant speed (as long as `a` is a constant). The smearing is evident near the peaks and troughs, and so a good first test of the solution would be the maximum absolute value between the exact and the numerical solution.

Add a CI/CD pipeline to run this test and validate each push commit. This lets us know if a specific submitted change damages the existing code. For this, fill in ./github/workflows/test.yml

<span style="color:green">JMS</span>.

<span style="color:red">Your workflow is missing a library (matplotlib). You can either include as a required file in setup.cgs, or import in the workflow or delete the dependencies of the library</span>.

> JB: This should now be in order!

## 7 – Resolution increase in space and time. 

Let us repeat the calculation of the previous paragraphs, increasing the number of space intervals by factors of 2. Check if the numerical solution gets increasingly close to the analytical solution. Important: the comparison must be made _for the same values of $x$ and $t$ for all resolutions you check_. Choose a fixed time when the traveling function has already gone through the whole domain a few times. 

In [ ]:
if __name__ == "__main__":
    check_time = 52
    nr_increases = 5    # number of times increasing space resolution
    b2s = 6     # start number of intervals in base 2
    nints = np.logspace(b2s,b2s+nr_increases,nr_increases+1,base=2, dtype=int)
    x0 = -2.6
    xf = 2.6
    a = -1
    maxabserr = np.zeros(len(nints))
    for i,nint in enumerate(nints):
        xx, dx = utils.get_xx(nint, x0, xf)
        nt = int((check_time+10)/dx[1])  # Set number of times steps to evolve enough to reach the desired check_time
        uu_t0 = get_uu_t0(xx)
        tt, uunt = nm.evolv_adv_burgers(xx, uu_t0,nt=nt,a=a,ddx = lambda x,u: nm.deriv_dnw(x,u,method="roll"), bnd_limits=[0,1])
        id_time_check = np.argmin(np.abs(tt-check_time))    # Get index of time closes to check_time
        u_exact_check = get_u_exact(xx,tt[id_time_check],a)
        maxabserr[i] = np.max(np.abs(uunt[id_time_check]-u_exact_check)) # Max error evaluated at desired time

In [ ]:
if __name__ == "__main__":
    fig, (ax,logax) = plt.subplots(figsize=(14,7),ncols=2, sharex=True)
    nints_inv_sqrt = (nints)**(-0.5)
    nints_inv = 1/nints
    ax.semilogx(nints, nints_inv_sqrt*np.max(maxabserr)/np.max(nints_inv_sqrt), label=r"$\propto nint^{-0.5}$")
    ax.semilogx(nints, maxabserr,"-x", label="error")
    logax.loglog(nints, maxabserr,"-x", label="error")
    logax.loglog(nints, nints_inv_sqrt*np.max(maxabserr)/np.max(nints_inv_sqrt), label=r"$\propto nint^{-0.5}$")
    logax.loglog(nints, nints_inv*np.min(maxabserr)/np.min(nints_inv), label=r"$\propto nint^{-1}$")
    for x in [ax,logax]:
        x.set_xticks(nints, [f"{val:d}" for val in nints])
        x.set_ylabel("Max absolute error")
        x.set_xlabel("nint")
        x.legend()
        x.set_title(f"t={tt[id_time_check]:.2f}");
    logax.yaxis.tick_right()
    logax.yaxis.set_label_position("right")

### Comments
Here I loop over a few different resolutions in space, and evaluate the maximum absolute error between the exact and the numerical solution at a desired time. The solution moves with the constant speed `a=-1`, and with an interval of length $5.2$ that should mean the function moves once through the domain in $\Delta t = 5.2$. I chose `time_check=52`, where each solution has passed through the interval 10 times. I think this is an effective test, and we can see the solution approaching the exact solution more and more for increased number of intervals in space. At `nint=1024` the max error is below $10\%$. I will try to implement this test as a CI/CD pipeline, in `test_ex_2b.py` from the `nm_lib` library.

As a side note I thought the slope resembled the invers square root function, so I just included that curve for comparison. This is not a proper test of the dependency of the space resolution, but I thought it was fun so I kept it for a reference.

<span style="color:green">JMS</span>.

<span style="color:blue">Great job.</span>.

<span style="color:Yellow">Could you plot in loglog scale. This will allow us to see the slope better. One important thing here is that it is not order 2 as the spatial derivative. And this has to do with the time-step scheme that it is not second order. </span>.

> JB: I've included a loglog scale version. As we can see it doesn't follow a straight curve, and of course most importantly not an order 2 curve. It's closer to "order one half", and for higher number of grid points it approaches a first order method. With a higher number of grid points the error caused by the discretization is reduced.